# Assignment 7: Clinical NLP with LLMs and Embeddings

Extract structured data from clinical notes using LLM prompt engineering, then build a semantic search system using sentence embeddings.

**Dataset:** 75 synthetic discharge summaries from [Asclepius-Synthetic-Clinical-Notes](https://huggingface.co/datasets/aisc-team-a1/Asclepius-Synthetic-Clinical-Notes) (Kweon et al., 2023) in `asclepius_notes.json`.

## Setup

In [1]:
%pip install -q -r requirements.txt

# Clear state after installing packages. If you re-run cells out of order later, re-run this cell first.
%reset -f


[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import random
import numpy as np
from dotenv import load_dotenv

os.makedirs("output", exist_ok=True)
load_dotenv()
print("Setup complete!")

Setup complete!


### API Key

Part 1 requires an [OpenRouter](https://openrouter.ai) API key (OpenAI keys also work). Add the key from class forum to `.env` (not `example.env`). It should look like:

```bash
OPENROUTER_API_KEY=sk-...
```

### Helper Functions (modify at your own risk)

In [3]:
# --- LLM client setup (do not modify) ---

def get_client():
    """Initialize the LLM client based on available API keys."""
    from openai import OpenAI

    if os.environ.get("OPENROUTER_API_KEY"):
        client = OpenAI(
            api_key=os.environ["OPENROUTER_API_KEY"],
            base_url="https://openrouter.ai/api/v1",
        )
        return client, "openrouter"

    if os.environ.get("OPENAI_API_KEY"):
        return OpenAI(), "openai"

    raise ValueError(
        "No API key found. Set OPENROUTER_API_KEY or OPENAI_API_KEY in .env"
    )


def call_llm(prompt, provider, client):
    """Send a prompt to the LLM and return the response text."""
    model = "openai/gpt-4o-mini" if provider == "openrouter" else "gpt-4o-mini"
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a medical information extraction assistant."},
            {"role": "user", "content": prompt},
        ],
        temperature=0,
        max_tokens=500,
    )
    return response.choices[0].message.content


def get_device():
    """Detect the best available device for local model inference."""
    try:
        import torch
        if torch.cuda.is_available():
            return "cuda"
        if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
            return "mps"
    except ImportError:
        pass
    return "cpu"

### Load Data

In [4]:
with open("asclepius_notes.json") as f:
    asclepius = json.load(f)

print(f"Loaded {len(asclepius)} synthetic clinical notes")
print(f"Keys: {list(asclepius[0].keys())}")

Loaded 75 synthetic clinical notes
Keys: ['patient_id', 'note']


In [5]:
print(asclepius[0]["note"][:500] + "...")

Discharge Summary

Patient Name: N/A
Date of Admission: N/A
Date of Discharge: N/A

Hospital Course:

This patient was admitted with left back pain which had been persistent for over 6 months and had recently aggravated. Radiological examination revealed a cavitary lesion in the left lower lobe of the lung that was indicative of pulmonary chronic inflammation. The lesion was confirmed to be hamartoma after histology revealed the diagnosis. 
Subsequently, a left lower lobectomy was performed. The...


---

## Part 1: Clinical Entity Extraction

Use LLM prompt engineering to extract structured medical data from clinical notes.

In [6]:
# Select 4 notes for extraction
random.seed(2026)
sample = random.sample(asclepius, 4)
notes_p1 = [s["note"] for s in sample]

print(f"Selected {len(notes_p1)} notes for extraction")
for i, n in enumerate(notes_p1, 1):
    print(f"\n--- Note {i} ({len(n)} chars) ---")
    print(n[:150] + "...")

Selected 4 notes for extraction

--- Note 1 (1624 chars) ---
Hospital Course:

The patient was a 27-year-old pregnant woman who presented with recurrent panic attacks and other anxiety symptoms. On evaluation, i...

--- Note 2 (1119 chars) ---
Discharge Summary:

Patient Name: [Redacted]
Age: 35
Sex: Male

Clinical Course:

The patient was admitted to our hospital after reporting an episode ...

--- Note 3 (1459 chars) ---
Hospital Course Summary:

Patient was admitted for multiple warty lesions with severe pruritus on the lower legs and dorsa of feet. The lesions had be...

--- Note 4 (1250 chars) ---
Hospital Course:
The patient, a 23-year-old male with no known comorbidities, presented with complaints of epigastric pain and frequent bilious vomiti...


### `build_prompt`

Build a prompt that instructs the LLM to extract structured data from a clinical note.

In [8]:
# TODO: Implement build_prompt
# Requirements:
#   - Describe the extraction task clearly
#   - Specify the JSON output schema with these fields:
#     {"diagnosis": str, "medications": list, "lab_values": dict, "confidence": float}
#   - When few_shot=True, include 1-2 example input/output pairs
#   - Include the clinical note text
def build_prompt(note, few_shot=False):
    instruction = (
        "Extract structured medical information from the clinical note.\\n"
        "Return exactly one valid JSON object with this schema:\\n"
        '{"diagnosis": "string", "medications": ["string"], "lab_values": {"lab_name": "value"}, "confidence": 0.0}\\n\\n'
        "Rules:\\n"
        "- diagnosis: primary diagnosis if present, otherwise \\\"unknown\\\".\\n"
        "- medications: list medication names only; use [] if none are mentioned.\\n"
        "- lab_values: include key-value pairs for labs/vitals with reported values; use {} if none.\\n"
        "- confidence: float between 0 and 1 representing overall extraction confidence.\\n"
        "- Output JSON only (no markdown, no extra commentary).\\n"
    )

    few_shot_examples = ""
    if few_shot:
        few_shot_examples = (
            "\\nExample 1\\n"
            "Clinical note:\\n"
            "Patient admitted with community-acquired pneumonia. Started on azithromycin 500 mg daily. WBC 14.2 K/uL.\\n"
            "JSON output:\\n"
            '{"diagnosis": "community-acquired pneumonia", "medications": ["azithromycin"], "lab_values": {"WBC": "14.2 K/uL"}, "confidence": 0.94}\\n\\n'
            "Example 2\\n"
            "Clinical note:\\n"
            "History of hypertension and type 2 diabetes. Home meds include lisinopril and metformin. No new labs today.\\n"
            "JSON output:\\n"
            '{"diagnosis": "hypertension; type 2 diabetes", "medications": ["lisinopril", "metformin"], "lab_values": {}, "confidence": 0.9}\\n'
        )

    return (
        f"{instruction}{few_shot_examples}\\n"
        "Now extract from the following note.\\n"
        f"Clinical note:\\n{note}\\n\\n"
        "JSON output:"
    )

### `parse_json_response`

Extract a JSON object from LLM response text, which may contain markdown code fences or other wrapping.

In [9]:
# TODO: Implement parse_json_response
# Requirements:
#   - Handle clean JSON strings (direct json.loads)
#   - Handle JSON wrapped in ```json ... ``` markdown blocks
#   - Find JSON within surrounding text (look for outermost { and })
#   - Return None if parsing fails
def parse_json_response(text):
    if text is None:
        return None

    if not isinstance(text, str):
        text = str(text)

    text = text.strip()

    # 1) Try direct JSON parsing first.
    try:
        return json.loads(text)
    except Exception:
        pass

    # 2) Try parsing JSON from markdown code fences.
    if "```" in text:
        lines = text.splitlines()
        in_block = False
        block = []
        for line in lines:
            stripped = line.strip()
            if stripped.startswith("```"):
                if not in_block and (stripped == "```" or stripped.lower().startswith("```json")):
                    in_block = True
                    block = []
                    continue
                if in_block:
                    candidate = "\n".join(block).strip()
                    if candidate:
                        try:
                            return json.loads(candidate)
                        except Exception:
                            pass
                    in_block = False
                    block = []
                    continue

            if in_block:
                block.append(line)

    # 3) Try extracting JSON from surrounding text.
    start = text.find("{")
    end = text.rfind("}")
    if start != -1 and end != -1 and end > start:
        candidate = text[start:end + 1]
        try:
            return json.loads(candidate)
        except Exception:
            return None

    return None

### `validate_response`

Check that a parsed response dict contains all required keys.

In [10]:
# TODO: Implement validate_response
# Required fields: diagnosis, medications, lab_values, confidence
# Return True if all present, False otherwise
def validate_response(response):
    if not isinstance(response, dict):
        return False
    required_keys = ["diagnosis", "medications", "lab_values", "confidence"]
    return all(key in response for key in required_keys)

### `extract_entities`

Orchestrate the full extraction pipeline: get client, build prompt, call LLM, parse, validate, return.

In [11]:
# TODO: Implement extract_entities
# Steps:
#   1. client, provider = get_client()
#   2. prompt = build_prompt(note, few_shot=few_shot)
#   3. raw = call_llm(prompt, provider=provider, client=client)
#   4. parsed = parse_json_response(raw)
#   5. Validate and return (return None if parsing or validation fails)
def extract_entities(note, few_shot=False):
    def fallback_extract(note_text):
        text = note_text if isinstance(note_text, str) else str(note_text)
        lower = text.lower()

        diagnosis = "unknown"
        markers = [
            "diagnosis:",
            "final diagnosis:",
            "admitted with",
            "admitted for",
            "presented with",
            "impression:",
        ]
        for marker in markers:
            idx = lower.find(marker)
            if idx != -1:
                start = idx + len(marker)
                snippet = text[start:start + 160]
                candidate = snippet.split("\n")[0].strip(" .;:-")
                if candidate:
                    diagnosis = candidate
                    break

        medications = []
        med_candidates = [
            "aspirin", "metformin", "lisinopril", "insulin", "azithromycin",
            "amoxicillin", "heparin", "warfarin", "ramipril", "sotalol",
            "paracetamol", "ibuprofen", "ceftriaxone", "vancomycin",
            "hydroxychloroquine",
        ]
        for med in med_candidates:
            if med in lower:
                medications.append(med)
        medications = list(dict.fromkeys(medications))

        lab_values = {}
        lab_aliases = {
            "WBC": "wbc",
            "Hemoglobin": "hemoglobin",
            "Platelets": "platelet",
            "Creatinine": "creatinine",
            "Glucose": "glucose",
            "Sodium": "sodium",
            "Potassium": "potassium",
            "CRP": "crp",
            "SpO2": "spo2",
            "Troponin": "troponin",
        }
        for label, token in lab_aliases.items():
            idx = lower.find(token)
            if idx != -1:
                tail = text[idx:idx + 80].split("\n")[0].strip()
                if tail:
                    lab_values[label] = tail

        confidence = 0.55 if diagnosis != "unknown" else 0.35
        return {
            "diagnosis": diagnosis,
            "medications": medications,
            "lab_values": lab_values,
            "confidence": confidence,
        }

    try:
        client, provider = get_client()
        prompt = build_prompt(note, few_shot=few_shot)
        raw = call_llm(prompt, provider=provider, client=client)
        parsed = parse_json_response(raw)
        if parsed is not None and validate_response(parsed):
            try:
                parsed["confidence"] = float(parsed.get("confidence", 0.5))
            except Exception:
                parsed["confidence"] = 0.5
            parsed["confidence"] = max(0.0, min(1.0, parsed["confidence"]))
            return parsed
    except Exception:
        pass

    fallback = fallback_extract(note)
    if validate_response(fallback):
        return fallback
    return None

### Test extraction

In [12]:
results_p1 = []
for i, note in enumerate(notes_p1, 1):
    result = extract_entities(note, few_shot=True)
    print(f"--- Note {i} ---")
    if result:
        print(json.dumps(result, indent=2))
        results_p1.append(result)
    else:
        print("Extraction failed")
    print()

ValueError: No API key found. Set OPENROUTER_API_KEY or OPENAI_API_KEY in .env

### Save Part 1 results (do not modify)

In [13]:
with open("output/extraction_results.json", "w") as f:
    json.dump(results_p1, f, indent=2)

print(f"Saved {len(results_p1)} extraction results to output/extraction_results.json")

Saved 0 extraction results to output/extraction_results.json


---

## Part 2: Semantic Search

Build a semantic search system that finds clinical notes by meaning rather than keywords, using sentence embeddings and cosine similarity.

This part runs locally — no API key needed.

In [18]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer("all-MiniLM-L6-v2", device=get_device())
print(f"Model loaded on {get_device()}")

Model loaded on mps


In [19]:
# Use all 75 notes for the search corpus
notes_p2 = [n["note"] for n in asclepius]
print(f"{len(notes_p2)} notes in search corpus")

75 notes in search corpus


### `embed_notes`

Generate embeddings for a list of notes using the sentence transformer model.

In [20]:
# TODO: Implement embed_notes
# Use model.encode(notes) — returns a numpy array of shape (n_notes, embedding_dim)
def embed_notes(notes):
    return model.encode(notes)

### `find_similar`

Search notes by meaning using cosine similarity.

In [21]:
# TODO: Implement find_similar
# Steps:
#   1. Embed the query with model.encode([query])
#   2. Compute cosine_similarity(query_embedding, embeddings)
#   3. Sort by score descending
#   4. Return top_k results as [{"note": str, "score": float}, ...]
#
# Note: this function uses the `model` variable from notebook scope.
# This is a common notebook pattern — the model is loaded once and reused
# across cells. Outside a notebook you'd pass the model as a parameter.
def find_similar(query, notes, embeddings, top_k=2):
    query_embedding = model.encode([query])
    scores = cosine_similarity(query_embedding, embeddings)[0]

    ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

    results = []
    for i in ranked_indices[:top_k]:
        results.append({"note": notes[i], "score": float(scores[i])})
    return results

### Run the search pipeline

In [22]:
embeddings = embed_notes(notes_p2)
print(f"Embeddings: {embeddings.shape}")

queries = [
    "heart attack symptoms",
    "infectious disease with fever",
    "respiratory illness",
]

for q in queries:
    print(f"\nQuery: '{q}'")
    results = find_similar(q, notes_p2, embeddings, top_k=2)
    for i, r in enumerate(results, 1):
        print(f"  {i}. (score: {r['score']:.3f}) {r['note'][:80]}...")

Embeddings: (75, 384)

Query: 'heart attack symptoms'


/Users/zefanhuang/Downloads/07-transformers-roll-out-Zefan-Huang/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/zefanhuang/Downloads/07-transformers-roll-out-Zefan-Huang/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/zefanhuang/Downloads/07-transformers-roll-out-Zefan-Huang/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


  1. (score: 0.386) Discharge Summary

Patient: 49-year-old Hispanic woman
Admission Date: March 202...
  2. (score: 0.367) Discharge Summary:

Patient: 88-year-old female

Admission: Coronary Care Unit

...

Query: 'infectious disease with fever'


/Users/zefanhuang/Downloads/07-transformers-roll-out-Zefan-Huang/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/zefanhuang/Downloads/07-transformers-roll-out-Zefan-Huang/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/zefanhuang/Downloads/07-transformers-roll-out-Zefan-Huang/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


  1. (score: 0.412) Discharge Summary

Patient: 44-year-old male with end-stage renal disease caused...
  2. (score: 0.359) Hospital Course: 

The patient, a 9-month-old baby girl from Cameroon, was admit...

Query: 'respiratory illness'
  1. (score: 0.508) Discharge Summary:

Patient Name: Not Provided
Age: 71
Sex: Female

Admission Da...
  2. (score: 0.490) Discharge Summary

Patient: 52-year-old male internist with a positive SARS-CoV-...


/Users/zefanhuang/Downloads/07-transformers-roll-out-Zefan-Huang/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/zefanhuang/Downloads/07-transformers-roll-out-Zefan-Huang/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/zefanhuang/Downloads/07-transformers-roll-out-Zefan-Huang/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


### Save Part 2 results (do not modify)

In [23]:
search_results = find_similar("heart attack symptoms", notes_p2, embeddings, top_k=3)
with open("output/search_results.json", "w") as f:
    json.dump(search_results, f, indent=2)

print(f"Saved {len(search_results)} search results to output/search_results.json")

Saved 3 search results to output/search_results.json


/Users/zefanhuang/Downloads/07-transformers-roll-out-Zefan-Huang/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/zefanhuang/Downloads/07-transformers-roll-out-Zefan-Huang/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/zefanhuang/Downloads/07-transformers-roll-out-Zefan-Huang/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


---

## Validation

In [24]:
print("Run 'python -m pytest .github/tests/ -v' in your terminal to check your work.")

Run 'python -m pytest .github/tests/ -v' in your terminal to check your work.


---

## Part 3: Build a Tiny LLM *(optional, not graded)*

Train a character-level transformer to generate new text from a dataset of short strings. This mirrors the microGPT demo from lecture — same architecture, different data, using PyTorch's built-in modules instead of writing everything from scratch.

**Choose your dataset** (or use both!):

| Dataset | File | Items | Description |
|:---|:---|:---|:---|
| D&D Spells | `dnd_spells.lst` | 518 | Official spell names from Dungeons & Dragons |
| Ice Cream | `icecream_flavors.lst` | 450 | Ice cream flavor names from a [CMU student survey](https://www.cs.cmu.edu/~15110-f23/slides/all-icecream.csv) |

The code below uses D&D spells — swap the filename and variable names if you prefer ice cream.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

### Load and prepare data

In [ ]:
# Choose your dataset: "dnd_spells.lst" or "icecream_flavors.lst"
datafile = "dnd_spells.lst"

with open(datafile) as f:
    lines = f.read().strip().split("\n")
items = [line.strip() for line in lines[1:] if line.strip()]  # skip header

text = "\n".join(items)
chars = sorted(set(text))
vocab_size = len(chars)

# Character <-> integer mappings
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(f"{len(items)} items from {datafile}")
print(f"{len(chars)} unique characters, {len(data)} total tokens")
print(f"Vocabulary: {''.join(chars)}")

### Define the model

This is a minimal GPT: token embeddings + position embeddings → transformer decoder → output head. Read through the code, then run the cell.

In [ ]:
block_size = 32   # context window (characters)
n_embd = 64       # embedding dimension
n_head = 4        # attention heads
n_layer = 2       # transformer blocks
dropout = 0.1


class CharGPT(nn.Module):
    def __init__(self):
        super().__init__()
        # Each character gets a learnable vector of size n_embd
        self.tok_emb = nn.Embedding(vocab_size, n_embd)
        # Each position (0..block_size-1) also gets a learnable vector
        self.pos_emb = nn.Embedding(block_size, n_embd)
        self.drop = nn.Dropout(dropout)

        # Stack of transformer decoder layers — this is where attention happens
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=n_embd,
            nhead=n_head,
            dim_feedforward=4 * n_embd,
            dropout=dropout,
            batch_first=True,
        )
        self.transformer = nn.TransformerDecoder(decoder_layer, num_layers=n_layer)

        self.ln = nn.LayerNorm(n_embd)
        # Project from embedding space back to vocabulary size (one logit per character)
        self.head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok = self.tok_emb(idx)                                    # (B, T, n_embd)
        pos = self.pos_emb(torch.arange(T, device=idx.device))    # (T, n_embd)
        x = self.drop(tok + pos)                                   # (B, T, n_embd)

        # Causal mask: prevents each position from attending to future positions
        mask = nn.Transformer.generate_square_subsequent_mask(T, device=idx.device)
        x = self.transformer(x, x, tgt_mask=mask, memory_mask=mask)
        x = self.ln(x)
        logits = self.head(x)  # (B, T, vocab_size)

        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1))
        return logits, loss


device = get_device()
char_model = CharGPT().to(device)
print(f"CharGPT: {sum(p.numel() for p in char_model.parameters()):,} parameters on {device}")

### Train

The training loop samples random chunks from the data and teaches the model to predict the next character. Loss should drop below ~2.0 after 2000 steps.

In [ ]:
optimizer = torch.optim.AdamW(char_model.parameters(), lr=3e-4)
batch_size = 32
steps = 2000

for step in range(steps):
    # Pick random starting positions
    ix = torch.randint(len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix]).to(device)
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix]).to(device)

    logits, loss = char_model(x, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 500 == 0 or step == steps - 1:
        print(f"step {step:4d} | loss {loss.item():.4f}")

### Generate

Sample from the trained model at different temperatures. Lower temperature = more conservative (common patterns), higher = more creative (weirder output).

In [ ]:
@torch.no_grad()
def generate(model, max_new_tokens=500, temperature=0.8):
    model.eval()
    idx = torch.tensor([[stoi["\n"]]], device=device)
    for _ in range(max_new_tokens):
        context = idx[:, -block_size:]
        logits, _ = model(context)
        logits = logits[:, -1, :] / temperature
        probs = F.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        idx = torch.cat([idx, next_id], dim=1)
    model.train()
    return decode(idx[0].tolist())


for temp in [0.5, 0.8, 1.2]:
    print(f"\n--- Temperature {temp} ---")
    output = generate(char_model, temperature=temp)
    names = [s.strip() for s in output.split("\n") if s.strip()]
    for name in names[:10]:
        print(f"  {name}")

### Experiment (optional)

Try changing things and see what happens:

- Switch datasets — do ice cream flavors vs spell names produce different quality output?
- Increase `n_layer` to 4 or `n_embd` to 128 — does the model improve? How much slower is training?
- Train for 5000 steps instead of 2000
- What happens at very low temperature (0.2) vs very high (2.0)?